# Chap 7. 필터링
* 영상의 필터링
* 블러링 : 영상 부드럽게 하기
* 샤프닝 : 영상 날카롭게 하기
* 잡음 제거 필터링

## 영상의 필터링
* 마스크 : 필터링의 성격을 정의하는 행렬 (=커널, 윈도우, 필터)
* 고정점 (anchor point) : 필터링 작업 수행 중인 기준 픽셀 위치 (행렬 정중앙)
    * 입력 영상의 (x, y) 기준으로 +- 1 위치씩 (3x3 마스크에서)
* 가장자리 픽셀 처리 : 특별한 처리 필요 (ex. **가상의 가장자리 픽셀 확장**)
     * **borderType** 이라는 열거형 상수(인자)로 가장자리 픽셀 처리 방법 선택  
       
* **cv2.filter2D**() : kernel을 이용해 **필터링해주는** 함수
> **dst = cv2.filter2D(src, ddepth, kernel, anchor, delta, borderType)**
* ddepth : 입력 영상의 깊이보다 더 큰 깊이로 설정해야 함 (overflow 방지)
    * **ddepth에 -1** 값을 지정하면 출력 영상 깊이 == 입력 영상 깊이
* anchor, delta, borderType은 디폴트값 있기 때문에 생략 가능

## 엠보싱 필터링
* 올록볼록 튀어나오는 느낌의 영상으로 변환
* 변화 적은 부분은 회색으로, 경계부분은 좀 더 밝거나 어둡게 설정
* [[-1, -1, 0], [-1, 0, 1], [0, 1, 1]] : 좌상단은 어둡게, 우하단은 밝게
* 보통 결과 영상에 128 더하는 것이 좋음 (for. 포화 연산에 의해 다 0 되는 것 방지)

In [1]:
# 엠보싱 필터링
import cv2
import numpy as np

src = cv2.imread('lenna.bmp', cv2.IMREAD_GRAYSCALE)

emboss = np.array([[-1, -1, 0], 
                   [-1, 0, 1], 
                   [0, 1, 1]], np.float32)

dst = cv2.filter2D(src, -1, emboss, delta=128) # (src, ddepth, kernel, delta)
# 꼭 dst로 따로 return값 받야아 함! (인자x)

cv2.imshow('src', src)
cv2.imshow('dst', dst)

cv2.waitKey()
cv2.destroyAllWindows()

## 블러링(Blurring) : 영상 부드럽게 하기
* 블러링 = 스무딩(smoothing)
* 거친 느낌의 영상을 부드럽게 만들거나, 잡음 제거 용으로도 사용 (전처리)

## 평균값 필터
* 특정 픽셀의 주변 픽셀들의 산술 평균으로 설정 ==> 급격한 변화 줄어들어 잡음 영향 줄음
* 필터 행렬의 모든 원소 값이 1로 설정 후, 전체 원소 개수로 각 요소들 나누기 (1/n)
    * **마스크의 크기가 커질수록** 더욱 부드러운 느낌의 영상 생성, But 연산량 크게 증가
    > **dst = cv2.blur()**

## (실습) 평균값 필터
* 평균 값 필터의 크기가 커질수록 부드러워지는 결과 영상 확인

In [4]:
def blurring_mean():
    src = cv2.imread('lenna.bmp', cv2.IMREAD_GRAYSCALE)
    
    cv2.imshow('src', src)
    
    for ksize in range(3, 9, 2): # 3, 5, 7 (kernel size)
        dst = cv2.blur(src, (ksize, ksize))
        
        desc = "Mean: %dx%d" % (ksize, ksize)
        cv2.putText(dst, desc, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1.0, 255, 1, cv2.LINE_AA)
        
        cv2.imshow('dst', dst)
        cv2.waitKey() # 아무 키 입력할 때마다 좌상단 text : 3x3 -> 5x5 -> 7x7
        
    cv2.destroyAllWindows()
    
blurring_mean() # 커널사이즈 클수록 블러링 심해짐

![img](blurring_mean.png)

## 가우시안 분포
* 평균을 중심으로 좌우 대칭의 종 모양을 갖는 확률 분포 (=정규 분포(normal distribution))
* 평균과 표준편차(시그마)에 따라 분포 모양 결정됨
    * **표준편차 : 평균으로부터 얼마나 떨어져 있는지**
* 영상의 경우, 평균 : 0인 가우시안 분포 함수 사용 (x=0에서 최댓값)
> 표준편차(시그마)가 작으면 ==> 뾰족한 형태  
> 펴준편차(시그마)가 크면 ==> 넓게 퍼지면서 완만한 형태
* 함수 값 : 특정 x가 발생할 수 있는 확률 ==> 그래프 아래 면적 합 : 1

## 가우시안 필터

* 가우시안 분포를 따르는 2차원 필터 마스크 행렬 ==> 평균 : (0,0), x축, y축의 표준편차는 각각 설정
* 평균 : (0,0) 이므로 그래프(**필터값**)는 **(0, 0) (중앙)위치에서 최댓값**, 평균에서 멀어질수록 함수 감소
* 보통 가우시안 필터 마스크의 크기 : (8 * 시그마 + 1)
> 가우시안 필터 마스크 이용한다 == **필터링 대상 픽셀(anchor, (0,0)) 근처에 가중치를 크게 준다** (가중평균 구하기위한 가중 행렬 역할)

* **표준 편차 값이 커질수록**, (8 * 시그마 + 1)에 따라 **마스크 크기도 함께 커지므로** 연산 속도 측면에서 부담이 될 수 있다
    * 2차원을 1차원 가우시안 분포 함수의 곱으로 분리해 연산 가능!
    * x축방향으로의 함수(9x1) & y축 방향으로의 함수(1x9)
    > 즉, **src * g * g.T (1차원 필터링 2번) == src * 2차원 필터링**  
    > (g : 시그마=1.0인 1x9 크기의 가우시안 마스크 행렬)
* 픽셀 하나당 연산 횟수 비교
     * 1차원 필터링 2번 (연산횟수 : 18번)
     * 2차원 필터링 1번 (연삿횟수 : 81번)

* **dst = cv2.GaussianBlur(src, ksize, sigmaX, sigmaY, borderType)**
> 만약 sigmaY = 0이면 sigmaX와 같은 값 사용

* cv2.getGaussianKernel() 함수를 사용해서 1차원 가우시안 필터 마스크 생성할 수도 있음 (사용자가 지정한 표준편차를 따르는 1차원 가우시안 필터 마스크 행렬. x축, y축 각각)

## (실습) 가우시안 필터
* 가우시안 표준 편차를 1부터 5까지 정수 단위로 증가시키면서 가우시안 필터링 수행
    * 표준편차 값이 커질수록 영상이 더욱 부드럽게 변함

In [5]:
def blurring_gaussian():
    src = cv2.imread('lenna.bmp', cv2.IMREAD_GRAYSCALE)
    
    cv2.imshow('src', src)
    
    for sigma in range(1, 6):
        # x와 y에 따른 sigma를 따로 지정하지 않고, 뒤의 sigma 인자에 따라 정해짐
        dst = cv2.GaussianBlur(src, (0, 0), sigma) # (src, (sigmaX, sigmaY), sigma)
        
        desc = "Gaussian: sigma = %d" % (sigma)
        cv2.putText(dst, desc, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1.0, 255, 1, cv2.LINE_AA)
        
        cv2.imshow('dst', dst)
        cv2.waitKey()
        
    cv2.destroyAllWindows()
    
blurring_gaussian()

![img](blurring_gaussian.png)